In [ ]:
import dask.dataframe as dd

import panel as pn
pn.extension("perspective")
pn.config.sizing_mode="stretch_width"

import holoviews as hv
hv.extension("bokeh")
import hvplot.dask 
import hvplot.pandas
import numpy as np

from config import TRAINING_GOLD_PATH

# Noaa Observations

In [ ]:
df = dd.read_parquet(TRAINING_GOLD_PATH)
df.head()

In [ ]:
len(df)

In [ ]:
def plot_bars(kdim="year", vdim="temperature", plot_type=hv.Bars):
    plot = plot_type(df, kdims=[kdim], vdims=vdim).aggregate(function=np.mean)
    plot.opts(height=500, responsive=True)
    
    load = hv.Curve(df, kdims=[kdim], vdims="load").aggregate(function=np.mean).opts(height=500, responsive=True)
    return (plot + load).cols(1)
plot_bars(plot_type=hv.Curve)

In [ ]:
kdims = ["year", "quarter", "day_of_year", "month", "day", "hour", "day_of_year",]
vdims = ["temperature", "dewpoint", "pressure", "wind_direction", "wind_speed", "sky_condition", "precipitation_one_hour", "precipitation_six_hour"]

In [ ]:
plots_bars = {}
for k in kdims[0:3]:
    for v in vdims[0:2]:
        plots_bars[(k,v)]=plot_bars(k,v, hv.Curve)

In [ ]:
hv.NdLayout(plots_bars, kdims=["key", "value"]).cols(1)

In [ ]:
selection_vdim = pn.widgets.Select(value=vdims[0], options=vdims)
selection_vdim

In [ ]:
container = pn.Column()

@pn.depends(kdim=selection_kdim, vdim=selection_vdim)
def plot_bars(kdim="year", vdim="temperature"):
    container.loading=True
    plot = hv.Bars(df_noaa, kdims=[kdim], vdims=vdim).aggregate(function=np.mean)
    plot.opts(height=500, responsive=True)
    container.loading=False
    return plot
container[:]=[selection_kdim, selection_vdim, plot_bars]
container

Previous: [Transform Ercot Loads](transform_ercot_loads.ipynb)
Next: [Train Model](train_model.ipynb)